In [5]:
%load_ext autoreload
%autoreload 2

from torchvision import models, transforms
from meanshift import *

MVTEC_PATH="../../../../nas-files/mvtec"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = models.efficientnet_b4(pretrained=True).features
net = net.to(device)
net = net.eval()

device

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'cuda'

In [93]:
log = []
layers = [5, 6]
X_, X_valid_, X_test_, X_labels_, T = dataloader(1, P=224, s=224, label_per_patch=False, load_size=256, crop_size=224, normalize=False, MVTEC_PATH=MVTEC_PATH )
X = concat_features(X_, net, blocks=layers, fmap_pool=False)
mu, means, cov, T = estimate_statistics(X[np.random.permutation(len(X))][:], mode="ledoit", BSZ=(5, 5), rho=2)
cov_inv = np.linalg.inv(cov)
L = torch.linalg.cholesky(torch.from_numpy(cov))  

In [ ]:
mu = torch.from_numpy(mu).to(device).float()
means = torch.from_numpy(means).to(device).float()
X = torch.from_numpy(X).to(device).float()
L = L.to(device).float()

In [39]:
local_hotelling_experimental(X[:10], T, mu, means, L, device=device)

tensor([881245.2500, 875501.9375, 881651.0000, 852389.5625, 878424.0000,
        862249.7500, 886882.7500, 828659.6250, 842410.5625, 847853.3750],
       device='cuda:0')

In [13]:
%timeit local_hotelling(X[:100], T, mu, means, cov_inv, BSZ=(5, 5), rho=2)

6.37 s ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
mu = torch.from_numpy(mu).to(device).float()
means = torch.from_numpy(means).to(device).float()
X = torch.from_numpy(X).to(device).float()
L = L.to(device).float()
%timeit local_hotelling_experimental(X[:10], T, mu, means, L, device=device)

8.64 ms ± 3.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
net = net.cuda()
%timeit concat_features(X_[:10], net, blocks=[5], fmap_pool=False)

12.3 ms ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
net = net.cuda()
x = torch.from_numpy( X_[:1] ).to(device)
nett = net[:4]
%timeit nett(x)

5.01 ms ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
net = net.cpu()
%timeit concat_features(X_[:10], net, layers=[6], fmap_pool=False)

360 ms ± 4.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit estimate_statistics(X[np.random.permutation(len(X))][:], mode="ledoit", BSZ=(5, 5), rho=2)

21.2 s ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit local_hotelling(X, T, mu, means, cov, BSZ=(5, 5), rho=2)

18.3 s ± 37.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit torch.mm(torch.ones((209*100, 6800)).to(device) , torch.ones((209*100, 6800)).to(device).T)

328 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit ledoit_wolf(np.ones((209*100, 6800)), assume_centered=False, block_size=1000)[0]

32.4 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%timeit np.cov(np.ones((209*100, 6800)).T)

4.87 s ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit local_hotelling_opt(X[:100], T, mu, means, cov_inv, device="cpu")

3.29 s ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit local_hotelling(X[:100], T, mu, means, cov_inv, BSZ=(5, 5), rho=2)

6.76 s ± 96.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
layers = [6]
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

X, X_valid, X_test = Cifar10_OneClass(train_classes=[9], balance=False, download=False, transform=transform)[0]
X, X_valid, X_test = asreshape(X, X_valid, X_test, shape=(3, 32, 32))
X = concat_features(X, net, blocks=layers, fmap_pool=False)
X_valid = concat_features(X_valid, net, blocks=layers, fmap_pool=False)
X_test = concat_features(X_test, net, blocks=layers, fmap_pool=False)

mu, means, cov_inv, T = estimate_statistics(X, mode="ledoit", BSZ=(1, 1), rho=1)

In [4]:
local_hotelling(X_valid, T, mu, means, cov_inv, BSZ=(1, 1), rho=1)[-5:]

array([195.39021016, 149.65624618, 225.99998571, 154.62704366,
       212.80977549])

In [9]:
local_hotelling_opt(torch.from_numpy(X_valid), T, torch.from_numpy(mu), torch.from_numpy(means), torch.from_numpy(cov_inv).float(), BSZ=(1, 1), rho=1)[-5:]

tensor([195.3902, 149.6562, 226.0000, 154.6270, 212.8098])

In [10]:
L = torch.linalg.cholesky(torch.from_numpy(cov_inv)).float()
local_hotelling_extreme(torch.from_numpy(X_valid), T, torch.from_numpy(mu), torch.from_numpy(means), L, BSZ=(1, 1), rho=1)[-5:]

tensor([195.3902, 149.6562, 226.0000, 154.6270, 212.8097])

In [7]:
s_in = local_hotelling(X_valid, T, mu, means, cov_inv, BSZ=(1, 1), rho=1)
s_out = local_hotelling(X_test, T, mu, means, cov_inv, BSZ=(1, 1), rho=1)

auc_local = roc_auc_score([0] * len(X_valid) + [1] * len(X_test), np.concatenate([s_in, s_out]))
auc_local

0.802433

In [12]:
s_in = local_hotelling_opt(torch.from_numpy(X_valid), T, torch.from_numpy(mu), torch.from_numpy(means), torch.from_numpy(cov_inv).float(), BSZ=(1, 1), rho=1)
s_out = local_hotelling_opt(torch.from_numpy(X_test), T, torch.from_numpy(mu), torch.from_numpy(means), torch.from_numpy(cov_inv).float(), BSZ=(1, 1), rho=1)

auc_local = roc_auc_score([0] * len(X_valid) + [1] * len(X_test), np.concatenate([s_in, s_out]))
auc_local

0.8024331666666666

In [13]:

s_in = local_hotelling_extreme(torch.from_numpy(X_valid), T, torch.from_numpy(mu), torch.from_numpy(means), L, BSZ=(1, 1), rho=1)
s_out = local_hotelling_extreme(torch.from_numpy(X_test), T, torch.from_numpy(mu), torch.from_numpy(means), L, BSZ=(1, 1), rho=1)

auc_local = roc_auc_score([0] * len(X_valid) + [1] * len(X_test), np.concatenate([s_in, s_out]))
auc_local

0.8024330555555557